In [1]:
import pandas as pd
import string
import regex as re
from pyvi import ViTokenizer
import emoji
import os

In [2]:
teencode_dict = {
    'ko': 'không', 'k': 'không', 'kg': 'không', 'khg': 'không', 'hok': 'không', 'hong': 'không', 'hem': 'không', 'hum': 'không', '0': 'không',
    'kh': 'không', 'kô': 'không', 'hok': 'không', 'hông': 'không',
    'dc': 'được', 'dk': 'được', 'dx': 'được', 'đc': 'được',
    'vs': 'với', 'wa': 'quá', 'wá': 'quá',
    'j': 'gì', 'z': 'gì', 'ji': 'gì',
    'bt': 'biết', 'biet': 'biết', 
    'ok': 'tốt', 'okie': 'tốt', 'oke': 'tốt', 'okê': 'tốt', 'oki': 'tốt', 'okla': 'tốt',
    'thanks': 'cảm ơn', 'thank': 'cảm ơn', 'tks': 'cảm ơn',
    'fb': 'facebook', 'insta': 'instagram', 'ins': 'instagram',
    'rep': 'trả lời', 'reply': 'trả lời', 'ib': 'nhắn tin',
    'mn': 'mọi người',
    'ntn': 'như thế nào',
    'v': 'vậy', 'zậy': 'vậy', 'za': 'vậy',
    'mk': 'mình', 'm': 'mình',
    'b': 'bạn',
    'r': 'rồi', 'rr': 'rồi',
    'cx': 'cũng', 'cxung': 'cũng',
    'ch': 'cửa hàng', 'shop': 'cửa hàng',
    'sz': 'kích thước', 'size': 'kích thước',
    'qtqd': 'quá tuyệt vời', 'qá': 'quá',
    'lun': 'luôn',
    'h': 'giờ',
    'đt': 'điện thoại', 'dt': 'điện thoại',
    'j': 'gì', 'k': 'không', 'm': 'mình', 'qhe': 'quan hệ', 'ny': 'người yêu',
        'vl': 'vãi lắm', 'khum': 'không', 'ck': 'chồng', 'vk': 'vợ', 'phimsex': 'phim người lớn',
        'tqtq': 'thích quá trời quá đất', 'bt': 'bình thường', 'qlq': 'quá là quạo', 'dc': 'được',
        'mn': 'mọi người', 'mk': 'mật khẩu', 'add': 'thêm', 'rep': 'trả lời', 'ib': 'nhắn tin riêng',
        'bb': 'bạn bè', 'stt': 'status', 'cmt': 'bình luận', 'snsd': 'sao nói sai được', 'chz': 'chuyện',
        'bvs': 'băng vệ sinh', 'chs': 'chuyện', 'ckx': 'chắc', 'cvc': 'công việc', 'tk': 'tài khoản',
        'tks': 'cảm ơn', 'y/c': 'yêu cầu', 'hpbd': 'happy birthday', 'oh': 'ồ', 'ka': 'không ai',
        'cái nè': 'cái này', 'cbi': 'chuẩn bị', 'bn': 'bạn', 'vlq': 'vãi luôn quá', 'z': 'vậy',
        'hqua': 'hôm qua', 'hnay': 'hôm nay', 'hn': 'hôm nay', 'vch': 'vãi chưởng', 'cmnr': 'chuẩn cơm mẹ nấu',
        'thks': 'cảm ơn', 'plz': 'làm ơn', 'ms': 'mới', 'r': 'rồi', 'lmk': 'làm ơn biết',
        'dcmm': 'được chưa mày', 'okie': 'ổn', 'cn': 'cuối tuần', 'dc ch': 'được chứ', 'sml': 'sấp mặt luôn',
        'pp': 'tạm biệt', 'haha': 'cười lớn', 'lol': 'cười lớn', 'gt': 'giới thiệu', 'nx': 'nhận xét',
        'bl': 'bình luận', 'aiya': 'ồ', 'zui': 'vui', 'iu': 'yêu', 'vs': 'với', 'ms': 'mới',
        'zề': 'về', 'rùi': 'rồi', 'nỳ': 'này', 'tru': 'thật', 'h m': 'hỏi mà',
        'lm': 'làm', 'phờ': 'phê', 'á đù': 'bất ngờ', 'đhs': 'đéo hiểu sao', 'hihi': 'cười nhỏ',
        'tíh': 'tính', 'èo': 'khó chịu', 'trời': 'chúa', 'gato': 'ghen ăn tức ở', 'vlol': 'vãi cười',
        'ăn lol': 'ăn không ngon', 'cứt': 'rất tệ', 'chịch': 'quan hệ tình dục', 'địt': 'quan hệ tình dục',
        'dm': 'đồ ngu', 'vkl': 'vãi cực luôn', 'vcc': 'vãi cả cười', 'cmm': 'câm mồm', 'dcm': 'đồ ngu ngốc',
        'cc': 'vãi thật', 'súc vật': 'người xấu', 'óc chó': 'người ngốc nghếch', 'đầu bò': 'người ngu',
        'lợn': 'người ngốc', 'ngu lol': 'rất ngu', 'hâm': 'người không bình thường', 'điên': 'người không tỉnh táo',
        'bẩn': 'rất dơ', 'thối': 'rất hôi', 'lầy': 'rất hài', 'nhây': 'rất dai dẳng', 'kaka': 'haha',
        'ahihi': 'cười lớn', 'ahuhu': 'rất buồn', 'kute': 'dễ thương', 'xinh vl': 'rất xinh', 'đẹp trai vl': 'rất đẹp trai',
        'vl cmm': 'rất lắm', 'ngu vcc': 'rất ngu', 'gato vcc': 'ghen ăn tức ở vãi', 'cmm vl': 'rất nhiều',
        'ahihi đồ ngốc': 'cười vào người ngốc', 'hụt': 'thiếu hụt', 'rớt': 'thất bại', 'lỗi': 'sai sót',
        'kool': 'rất chất', 'phét': 'không thật', 'nói phét': 'nói dối', 'bốc phét': 'bịa chuyện', 'chém gió': 'nói phóng đại',
        'bựa': 'rất kỳ cục', 'đểu': 'rất xấu', 'đồ đểu': 'người xấu', 'khốn': 'rất tệ', 'vô đạo đức': 'rất xấu',
        'cực phẩm': 'rất tốt', 'bá đạo': 'rất giỏi', 'ngon lành': 'rất tốt', 'kém chất lượng': 'rất tệ', 'toang': 'thất bại',
        'xàm': 'không có ý nghĩa', 'xàm lol': 'rất nhảm nhí', 'nhảm': 'vô nghĩa', 'dỗi': 'giận dỗi', 'giận hờn': 'không vui',
        'hận': 'rất ghét', 'phũ': 'rất lạnh lùng', 'đào hoa': 'rất hấp dẫn', 'chảnh': 'kiêu căng', 'kiêu': 'rất tự cao',
        'sến': 'rất lãng mạn', 'ngôn tình': 'rất tình cảm', 'gớm': 'rất xấu', 'tởm': 'rất ghê', 'rùng mình': 'rất sợ',
        'tởm lol': 'rất kinh khủng', 'kinh tởm': 'rất ghê', 'dị': 'rất lạ', 'dị hợm': 'rất kỳ quái', 'quái': 'rất lạ',
        'quái đản': 'rất kỳ dị', 'quái dị': 'rất xấu', 'chán đời': 'rất buồn', 'tự tử': 'kết thúc cuộc sống',
        'mất tích': 'không thấy đâu', 'đi bụi': 'bỏ nhà đi', 'cờ bạc': 'đánh bạc', 'nghiện ngập': 'rất tệ', 'đập đá': 'sử dụng ma túy',
        'chơi bời': 'rất phóng túng', 'ăn chơi': 'rất hưởng thụ', 'bạo lực': 'rất mạnh', 'phang': 'đánh mạnh', 'vả': 'tát',
        'đấm': 'đánh', 'đá': 'chân đánh', 'chém': 'sử dụng dao', 'hành hung': 'đánh đập', 'ẩu đả': 'đánh nhau',
        'đánh lộn': 'đánh nhau', 'hỗn chiến': 'đánh nhau đông người', 'bạo loạn': 'rất hỗn loạn',
    'sp': 'sản phẩm', 'spham': 'sản phẩm',
    'ship': 'giao hàng', 'sip': 'giao hàng', 'shiper': 'người giao hàng', 'shipper': 'người giao hàng',
    'nv': 'nhân viên',
    'đg': 'đóng gói', 'donggoi': 'đóng gói',
    'sd': 'sử dụng',
    'bh': 'bảo hành',
    'auth': 'chính hãng', 'authentic': 'chính hãng', 'real': 'thật',
    'fake': 'giả',
    'check': 'kiểm tra',
    'ord': 'đặt hàng', 'order': 'đặt hàng',
    'feedback': 'phản hồi', 'review': 'đánh giá', 'rv': 'đánh giá',
    'bt': 'bình thường',
    'nh': 'nhanh',
    'chậm': 'chậm', 'chậm trễ': 'chậm', 'trễ': 'chậm',
    'chậm trễ': 'chậm', 'trễ hẹn': 'chậm', 'trễ hàng': 'chậm',
    'chậm hàng': 'chậm', 'trễ giao hàng': 'chậm', 'trễ thời gian': 'chậm',
    'chậm tiến độ': 'chậm', 'trễ tiến độ': 'chậm', 'trễ thời gian giao hàng': 'chậm',
    'chậm tiến độ giao hàng': 'chậm', 'trễ thời gian giao hàng': 'chậm',
}

In [7]:
abbreviation_dict = {
    'tp': 'thành phố',
    'hcm': 'hồ chí minh', 'sg': 'sài gòn',
    'hn': 'hà nội', 'đn': 'đà nẵng',
    'cty': 'công ty',
    'bs': 'bác sĩ',
    'bv': 'bệnh viện',
    'cs': 'chất liệu', 
    'ch': 'cấu hình', 
    'mnc': 'màn hình cong', 
    'hsd': 'hạn sử dụng',
    'hdsd': 'hướng dẫn sử dụng',
    'hd': 'hướng dẫn',
    
}

In [3]:
def normalize_for_sentiment(text):
    if not isinstance(text, str):
        return ""

    # 1. Lowercase
    text = text.lower()

    # 2. Loại bỏ dấu * và cụm từ "xem thêm"
    text = re.sub(r'\*', ' ', text)
    text = re.sub(r'\bxem thêm\b', ' ', text)

    # 3. Xử lý Teencode và Viết tắt
    combined_dict = {**abbreviation_dict, **teencode_dict}
    words = text.split()
    processed_words = [combined_dict.get(word, word) for word in words]
    text = ' '.join(processed_words)

    # 4. Chuyển đổi Emoji thành văn bản
    text = emoji.demojize(text, delimiters=(" EMOJI_", " "))

    # 5. Xử lý lỗi chính tả cơ bản (lặp ký tự)
    text = re.sub(r'([\p{L}])\1+', r'\1', text, flags=re.UNICODE)

    # 6. Loại bỏ dấu câu và ký tự đặc biệt không cần thiết
    text = re.sub(r'[^\p{L}\p{N}\s]', ' ', text, flags=re.UNICODE)

    # 7. Tách từ (Word Tokenization) - *** SỬ DỤNG pyvi ***
    try:
        
        text = ViTokenizer.tokenize(text)
        # Đảm bảo kết quả luôn là string
        if not isinstance(text, str): text = str(text)
    except Exception as e:
        print(f"Warning: Lỗi ViTokenizer.tokenize: {e} - Sử dụng split đơn giản cho: {text[:50]}...")
        # Fallback nếu pyvi gặp lỗi
        text = ' '.join(text.split())

    # *** KHÔNG LOẠI BỎ STOP WORDS ***

    # 8. Loại bỏ khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [4]:
csv_file_path = '../Labeled_Data/Simple_LabelWithoutOther.csv'  
text_column_name = 'content'        
output_column_name = 'content' 
output_csv_path = 'Simple_Label_Normalized_Sentiment_pyvi.csv' 
save_output = True

In [8]:
print(f"Đang xử lý file: {csv_file_path} (Sử dụng pyvi để tách từ)")

# Kiểm tra file tồn tại trước khi đọc
if not os.path.exists(csv_file_path):
    print(f"Lỗi: Không tìm thấy file tại đường dẫn '{csv_file_path}'")
else:
    try:
        # Đọc file CSV
        print("Đang đọc file CSV...")
        df = pd.read_csv(csv_file_path)
        print(f"Đọc file thành công. Số dòng: {len(df)}")

        # Kiểm tra xem cột văn bản có tồn tại không
        if text_column_name not in df.columns:
            print(f"Lỗi: Không tìm thấy cột '{text_column_name}' trong file CSV.")
        else:
            print(f"\nBắt đầu chuẩn hóa cột '{text_column_name}'...")
            # Áp dụng hàm chuẩn hóa, đảm bảo xử lý cả giá trị NaN bằng cách điền ''
            # Sử dụng .progress_apply nếu muốn có thanh tiến trình (cần cài tqdm: pip install tqdm)
            # Hoặc có thể dùng thư viện pandarallel nếu file lớn và muốn xử lý song song
            df[output_column_name] = df[text_column_name].fillna('').astype(str).apply(normalize_for_sentiment)
            print("Chuẩn hóa hoàn tất.")

            # Hiển thị một vài dòng kết quả để kiểm tra
            print("\nXem trước 5 dòng kết quả:")
            try:
                 pd.set_option('display.max_colwidth', 200) # Hiển thị nội dung cột dài hơn
                 pd.set_option('display.width', 1000) # Tăng chiều rộng hiển thị tổng thể
            except Exception as pd_option_err:
                 print(f"(Lưu ý nhỏ: Không thể đặt tùy chọn hiển thị pandas: {pd_option_err})")

            print(df[[text_column_name, output_column_name]].head())

            # (Tùy chọn) Lưu kết quả ra file CSV mới
            if save_output:
                try:
                    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig') # Dùng utf-8-sig để Excel đọc tiếng Việt tốt hơn
                    print(f"\nĐã lưu kết quả vào file: {output_csv_path}")
                except Exception as e_save:
                    print(f"\nLỗi khi lưu file CSV '{output_csv_path}': {e_save}")

    except Exception as e_main:
        print(f"\nĐã xảy ra lỗi không mong muốn trong quá trình xử lý: {e_main}")

Đang xử lý file: ../Labeled_Data/Simple_LabelWithoutOther.csv (Sử dụng pyvi để tách từ)
Đang đọc file CSV...
Đọc file thành công. Số dòng: 6469

Bắt đầu chuẩn hóa cột 'content'...
Chuẩn hóa hoàn tất.

Xem trước 5 dòng kết quả:
                                                                                                                                                                                                   content                                                                                                                                                                                                  content
0  sách rất hay dựa trên nghiên_cứu và có hướng_dẫn cách thực_hiện cụ_thể đọc rất lôi_cuốn và thực_tế một_vài chi_tiết mình thấy đã thực_hành rất hiệu_quả bạn nào muốn thay_đổi thói_quen bản_thân mà ...  sách rất hay dựa trên nghiên_cứu và có hướng_dẫn cách thực_hiện cụ_thể đọc rất lôi_cuốn và thực_tế một_vài chi_tiết mình thấy đã thực_hành rất hiệu_quả bạn nào muốn 